In [1]:
from pomegranate import *

In [2]:
def auto_complete(probability_table):
    for i in range(len(probability_table)):
        v = probability_table[i].copy()
        v[-2] = '-' + v[-2]
        v[-1] = 1 - v[-1]
        probability_table.append(v)
    return probability_table

In [3]:
burglary = DiscreteDistribution(dict(auto_complete([['b', 0.5]])))
earthquake = DiscreteDistribution(dict(auto_complete([['e', 0.002]])))
alarm = ConditionalProbabilityTable(
    auto_complete([
        ['b', 'e', 'a', 0.95],
        ['b', '-e', 'a', 0.94],
        ['-b', 'e', 'a', 0.29],
        ['-b', '-e', 'a', 0.001]
    ]),
    [burglary, earthquake]
)
john_calls = ConditionalProbabilityTable(
    auto_complete([
        ['a', 'j', 0.90],
        ['-a', 'j', 0.05]
    ]),
    [alarm]
)
mary_calls = ConditionalProbabilityTable(
    auto_complete([
        ['a', 'm', 0.70],
        ['-a', 'm', 0.01]
    ]),
    [alarm]
)

In [4]:
model = BayesianNetwork('Burglary')
nodes = [burglary, earthquake, alarm, john_calls, mary_calls]
states = [State(node) for node in nodes]
model.add_states(*states)
model.add_edge(states[0], states[2])
model.add_edge(states[1], states[2])
model.add_edge(states[2], states[3])
model.add_edge(states[2], states[4])
model.bake()

In [6]:
model.probability(['b', 'e', 'a', 'j', 'm'])

0.0005985000000000006